# AWS Purpose-built AI Services - Code Examples

This notebook contains code examples for all AWS Purpose-built AI services:
- Amazon Transcribe (Speech-to-Text)
- Amazon Polly (Text-to-Speech)
- Amazon Translate (Language Translation)
- Amazon Comprehend (Natural Language Processing)
- Amazon Rekognition (Image & Video Analysis)

**Note:** 
- This notebook supports loading configuration from a `.env` file
- Copy `.env.example` to `.env` and fill in your values
- Alternatively, you can use `aws configure` or environment variables


## Prerequisites

Before setting up, ensure you have the following installed:

- **Python 3.8+** - Required for running the notebook
- **uv** - Fast Python package installer and resolver ([Install uv](https://github.com/astral-sh/uv))
- **AWS Account** - With appropriate IAM credentials and permissions
- **AWS Credentials** - Configured via `.env` file, `aws configure`, or environment variables

## Setup Instructions

Before running this notebook, you need to set up the virtual environment and install dependencies:

### 1. Activate the virtual environment (Optional)

The virtual environment was created using `uv`. You can activate it manually, or use `uv run` which automatically uses the venv:

```bash
source .venv/bin/activate
```

**Note:** If you use `uv run` (Step 4), you don't need to activate the venv manually.

### 2. Install dependencies

Install all required packages using `uv pip install`:

```bash
uv pip install -r requirements.txt
```

### 2.5. Register the virtual environment as a Jupyter kernel (Important!)

Register the venv's Python as a Jupyter kernel so Jupyter Lab can use it:

```bash
uv run python -m ipykernel install --user --name=aws-ai-services --display-name "Python (AWS AI Services)"
```

**Note:** This registers the venv kernel so you can select it in Jupyter Lab. You only need to do this once.

### 3. Configure AWS credentials

Copy `.env.example` to `.env` and fill in your AWS credentials:

```bash
cp .env.example .env
# Edit .env with your actual AWS credentials
```

Alternatively, you can use `aws configure` or set environment variables.

### 4. Launch Jupyter Lab

Start Jupyter Lab using `uv run`. This automatically uses the virtual environment:

```bash
uv run jupyter lab
```

**Note:** `uv run` automatically uses the `.venv` virtual environment. When Jupyter Lab opens, select the "Python (AWS AI Services)" kernel from the kernel selector.

---


In [ ]:
# Install and import required libraries
import boto3
import json
import os
from dotenv import load_dotenv

# Load environment variables from .env file (if it exists)
# This will load variables from .env into os.environ
load_dotenv()

# Verify boto3 is installed
print(f"boto3 version: {boto3.__version__}")

# Initialize AWS session
# Priority: .env file > environment variables > ~/.aws/credentials > IAM role
session = boto3.Session(
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name=os.getenv('AWS_DEFAULT_REGION', 'us-east-1')
)

print(f"AWS Region: {session.region_name}")
if os.getenv('AWS_ACCESS_KEY_ID'):
    print("✓ Using credentials from .env file")
elif os.environ.get('AWS_ACCESS_KEY_ID'):
    print("✓ Using credentials from environment variables")
else:
    print("✓ Using default AWS credentials (~/.aws/credentials or IAM role)")


## 1. Amazon Transcribe: Speech-to-Text

Amazon Transcribe is an automatic speech recognition (ASR) service that makes it easy to add speech-to-text capabilities to applications.


In [ ]:
# Amazon Transcribe Example
# Using AWS SDK (Python boto3) to start a transcription job and get results

client = boto3.client('transcribe', region_name=session.region_name)
job_name = 'example-job'

# Get S3 URI from environment variables or use default
s3_bucket = os.getenv('S3_BUCKET', 'your-bucket-name')
s3_key = os.getenv('TRANSCRIBE_S3_KEY', 'input.wav')
job_uri = f's3://{s3_bucket}/{s3_key}'

# Output location for transcription results
output_bucket = os.getenv('TRANSCRIBE_OUTPUT_BUCKET', s3_bucket)  # Default to same bucket as input
output_key = os.getenv('TRANSCRIBE_OUTPUT_KEY', f'transcriptions/{job_name}.json')  # Output JSON file
output_uri = f's3://{output_bucket}/{output_key}'

# Get media format from environment variable or auto-detect from file extension
# Supported formats: mp3, mp4, wav, flac, ogg, amr, webm, m4a
media_format = os.getenv('TRANSCRIBE_MEDIA_FORMAT', None)
if not media_format:
    # Auto-detect format from file extension
    file_ext = s3_key.split('.')[-1].lower() if '.' in s3_key else 'wav'
    format_map = {
        'mp3': 'mp3', 'mp4': 'mp4', 'wav': 'wav', 'flac': 'flac',
        'ogg': 'ogg', 'amr': 'amr', 'webm': 'webm', 'm4a': 'm4a'
    }
    media_format = format_map.get(file_ext, 'wav')  # Default to wav if unknown

# Note: Set S3_BUCKET, TRANSCRIBE_S3_KEY, and optionally TRANSCRIBE_MEDIA_FORMAT in .env file
# Supported audio formats: mp3, mp4, wav, flac, ogg, amr, webm, m4a
# Uncomment the following lines to start a transcription job and get results:# 

# Start the transcription job
response = client.start_transcription_job(
    TranscriptionJobName=job_name,
    Media={'MediaFileUri': job_uri},
    MediaFormat=media_format,
    LanguageCode='en-US',
    OutputBucketName=output_bucket,  # Where to save transcription
    OutputKey=output_key  # Output file key
)
print('Transcription job started:', response['TranscriptionJob']['TranscriptionJobName'])

# Wait for the job to complete
import time
while True:
    status = client.get_transcription_job(TranscriptionJobName=job_name)
    job_status = status['TranscriptionJob']['TranscriptionJobStatus']
    if job_status in ['COMPLETED', 'FAILED']:
        break
    print(f"Job status: {job_status}. Waiting...")
    time.sleep(5)

# Print the output location
if job_status == 'COMPLETED':
    print(f"\nTranscription completed!")
    print(f"Transcription output: {output_uri}")
    print(f"You can download it from S3 or access it via the AWS Console.")
else:
    print(f"Transcription job failed: {status['TranscriptionJob'].get('FailureReason', 'Unknown error')}")

print(f"Example code ready. S3 URI: {job_uri}, Media Format: {media_format}")
print(f"Output will be saved to: {output_uri}")


## 2. Amazon Polly: Text-to-Speech

Amazon Polly is a service that turns text into lifelike speech using deep learning.


In [ ]:
# Amazon Polly Example
# Using AWS SDK (Python boto3) to synthesize speech

client = boto3.client('polly', region_name=session.region_name)
text = """Welcome to our automated customer service system. Your call is important to us.

If you need to file a new insurance claim, please press one. For questions about an existing claim, press two. To speak with a customer service representative, press three. For billing inquiries, press four.

All of our agents are currently busy. Please hold, and your call will be answered in the order it was received. The estimated wait time is approximately five minutes.

Thank you for your patience."""

response = client.synthesize_speech(
    Text=text,
    OutputFormat='mp3',
    VoiceId='Joanna'
)

# Save the audio file
with open('output.mp3', 'wb') as file:
    file.write(response['AudioStream'].read())

print(f"Audio file saved as 'output.mp3'")
print(f"Text synthesized: {text}")


## 3. Amazon Translate: Language Translation

Amazon Translate is a neural machine translation service for translating text between languages.


In [ ]:
# Amazon Translate Example
# Using AWS SDK (Python boto3) to translate text

client = boto3.client('translate', region_name=session.region_name)
text = """Thank you for contacting our customer service department. We have received your insurance claim and are processing it.
Your claim number is CL-2024-8923. Our team will review the documentation you provided, including the police report, medical records, and vehicle damage assessment.

We expect to complete our review within 5-7 business days. Once the review is complete, we will contact you via email or phone to discuss the next steps.

If you have any questions or need to provide additional information, please call us at 1-800-555-0123 between 9:00 AM and 6:00 PM Eastern Time, Monday through Friday. You can also visit our website at www.insurancecompany.com and log into your account to check the status of your claim.
We understand this is a stressful time, and we are here to help. Please don't hesitate to reach out if you need assistance.

Best regards,
Customer Service Team"""

response = client.translate_text(
    Text=text,
    SourceLanguageCode='en',
    TargetLanguageCode='es'
)

print(f"Original text: {text}")
print(f"Translated text: {response['TranslatedText']}")


## 4. Amazon Comprehend: Natural Language Processing

Amazon Comprehend is a natural language processing (NLP) service that uses machine learning to find insights and relationships in text.


In [ ]:
# Amazon Comprehend Example
# Using AWS SDK (Python boto3) to analyze sentiment and entities

client = boto3.client('comprehend', region_name=session.region_name)
text = """Hello, I need to file a claim. My name is Sarah Johnson. My policy number is AC-8923471. I had a car accident yesterday, June 15th, 2024, around 3:30 PM.

I was driving on Highway 101 near the intersection with Main Street in San Francisco, California. It was raining pretty hard, and I was going about 45 miles per hour in the right lane. A blue Honda Accord, license plate 7ABC123, came from behind and rear-ended my 2022 Toyota Camry. The driver was a man named Robert Martinez. He gave me his insurance information—he's with State Farm Insurance, policy number SF-4567890.

My car has significant damage to the rear bumper and trunk. The tailgate won't close properly, and there's some damage to the rear window. The other driver's car also has front-end damage. We called the police, and Officer Michael Chen from the San Francisco Police Department arrived. He filed a report, case number SF-2024-4872.

I'm okay, but I have some neck and back pain. I went to San Francisco General Hospital Emergency Room last night. They took X-rays and said I might have whiplash. I have a follow-up appointment with Dr. Emily Watson on Monday, June 17th.

My car is currently at Mike's Auto Body Shop on Market Street. They gave me an initial estimate of $8,500 for repairs, but they need to do a more detailed inspection. I also had to get a rental car from Enterprise Rent-A-Car. That's costing me about $45 per day.

I have full coverage with your company, Progressive Insurance. My deductible is $500. I really need to get my car fixed quickly because I use it for work—I'm a nurse at Kaiser Permanente Hospital and need to get to my shifts. Can someone please help me get this processed quickly? I'm really stressed out about this whole situation."""

# Detect sentiment
sentiment = client.detect_sentiment(Text=text, LanguageCode='en')
print('Sentiment:', sentiment['Sentiment'])
print('Sentiment Scores:', sentiment['SentimentScore'])

# Detect entities
entities = client.detect_entities(Text=text, LanguageCode='en')
print('\nEntities found:')
for entity in entities['Entities']:
    print(f"  Entity: {entity['Text']}, Type: {entity['Type']}, Confidence: {entity['Score']:.2f}")


## 5. Amazon Rekognition: Image & Video Analysis

Amazon Rekognition is a fully managed AWS service that enables image and video analysis using deep learning.


In [ ]:
# Amazon Rekognition Example
# Using AWS SDK (Python boto3) to analyze an image

client = boto3.client('rekognition', region_name=session.region_name)

# Get S3 configuration from environment variables or use defaults
s3_bucket = os.getenv('S3_BUCKET', 'your-bucket-name')  # Reuse same bucket as Transcribe
s3_image_key = os.getenv('REKOGNITION_S3_KEY', 'input.png')  # Image file key in S3

# Note: Set REKOGNITION_S3_KEY in .env file, or replace the value above
# Uncomment the following lines to analyze an image from S3:
response = client.detect_labels(
    Image={'S3Object': {'Bucket': s3_bucket, 'Name': s3_image_key}},
    MaxLabels=10,
    MinConfidence=70.0
)

print("Labels detected:")
for label in response['Labels']:
    print(f"  Label: {label['Name']}, Confidence: {label['Confidence']:.2f}%")

print(f"Example code ready. S3 bucket: {s3_bucket}, S3 image key: {s3_image_key}")
